In [ ]:
#SOLAR POWER PLANT ANALYSIS 


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math as math
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import fbprophet
from fbprophet import Prophet
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#https://www.kaggle.com/virosky/how-to-manage-a-solar-power-plant

In [ ]:
#IMPORTING ALL THE DATA FROM THE DATASETS
df1 = pd.read_csv('../input/solar-power-generation-data/Plant_1_Generation_Data.csv')
df2 = pd.read_csv('../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
df3 = pd.read_csv('../input/solar-power-generation-data/Plant_2_Generation_Data.csv')
df4 = pd.read_csv('../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv')

In [ ]:
#Splitting all the 4 dataframes' DATE_TIME into DATE and making it a datetime64 dtype


df1['DATE_TIME'] = pd.to_datetime(df1['DATE_TIME'],format = '%d-%m-%Y %H:%M')   #datetime 
df1['DATE'] = pd.to_datetime(df1['DATE_TIME'],format = '%d-%m-%Y %H:%M').dt.date   #split
df1['DATE'] = pd.to_datetime(df1['DATE'] )      #datetime series
df1['TIME'] = df1['DATE_TIME'].dt.time
df1['TIME'] = pd.to_datetime(df1['TIME'], format = '%H:%M:%S')
df1['HOUR'] = pd.to_datetime(df1['TIME'],format='%H:%M:%S').dt.hour
df1['MINUTES'] = pd.to_datetime(df1['TIME'],format='%H:%M:%S').dt.minute



df2['DATE_TIME'] = pd.to_datetime(df2['DATE_TIME'],format = '%Y-%m-%d %H:%M')   #datetime 
df2['DATE'] = pd.to_datetime(df2['DATE_TIME'],format = '%Y-%m-%d %H:%M').dt.date   #split
df2['DATE'] = pd.to_datetime(df2['DATE'] )      #datetime series
df2['TIME'] = df2['DATE_TIME'].dt.time
df2['TIME'] = pd.to_datetime(df2['TIME'], format = '%H:%M:%S')
df2['HOUR'] = pd.to_datetime(df2['TIME'],format='%H:%M:%S').dt.hour
df2['MINUTES'] = pd.to_datetime(df2['TIME'],format='%H:%M:%S').dt.minute



df3['DATE_TIME'] = pd.to_datetime(df3['DATE_TIME'],format = '%Y-%m-%d %H:%M')   #datetime 
df3['DATE'] = pd.to_datetime(df3['DATE_TIME'],format = '%Y-%m-%d %H:%M').dt.date   #split
df3['DATE'] = pd.to_datetime(df3['DATE'] )      #datetime series
df3['TIME'] = df3['DATE_TIME'].dt.time
df3['TIME'] = pd.to_datetime(df3['TIME'], format = '%H:%M:%S')
df3['HOUR'] = pd.to_datetime(df3['TIME'],format='%H:%M:%S').dt.hour
df3['MINUTES'] = pd.to_datetime(df3['TIME'],format='%H:%M:%S').dt.minute




df4['DATE_TIME'] = pd.to_datetime(df2['DATE_TIME'],format = '%Y-%m-%d %H:%M')   #datetime 
df4['DATE'] = pd.to_datetime(df2['DATE_TIME'],format = '%Y-%m-%d %H:%M').dt.date   #split
df4['DATE'] = pd.to_datetime(df2['DATE'] )      #datetime series
df4['TIME'] = df4['DATE_TIME'].dt.time
df4['TIME'] = pd.to_datetime(df4['TIME'], format = '%H:%M:%S')
df4['HOUR'] = pd.to_datetime(df4['TIME'],format='%H:%M:%S').dt.hour
df4['MINUTES'] = pd.to_datetime(df4['TIME'],format='%H:%M:%S').dt.minute




 
    
#Changing the incorrectly noted DC Power in the first file
for i in range(len(df1.index)) : 
    df1['DC_POWER'].iloc[i] = df1['DC_POWER'].loc[i]/10

In [ ]:
#NUMBER OF INVERTERS 
#Plant 1
print("Plant 1-", df1['SOURCE_KEY'].nunique())
#Plant 2
print("Plant 2-", df2['SOURCE_KEY'].nunique())

In [ ]:
#NAME OF THE INVERTERS IN PLANT 1
df1['SOURCE_KEY'].unique()

In [ ]:
#NAME OF THE INVERTERS IN PLANT 2
df3['SOURCE_KEY'].unique()

In [ ]:
#NAME OF THE WEATHER SESNOR INVERTERS 
print('PLANT 1',df2['SOURCE_KEY'].unique())
print('PLANT 2',df4['SOURCE_KEY'].unique())

In [ ]:
#MOST CONSISTENT INVERTER IN PLANT 1

a = df1.groupby('SOURCE_KEY')['DC_POWER'].sum()
powList = list()
source_list = df1['SOURCE_KEY'].unique()
source_valcounts = df1['SOURCE_KEY'].value_counts()
for i in range(0,22):
    counts = source_valcounts[source_list[i]]
    powList.append(a[i]/counts)
num = max(powList)
for o in range(0,22):
    if num == powList[o]:
        print("MOST CONSISTENT INVERTER IN PLANT 1 :", source_list[o])

In [ ]:
#MOST CONSISTENT INVERTER IN PLANT 2

a = df3.groupby('SOURCE_KEY')['DC_POWER'].sum()
powList = list()
source_list = df3['SOURCE_KEY'].unique()
source_valcounts = df3['SOURCE_KEY'].value_counts()
for i in range(0,22):
    counts = source_valcounts[source_list[i]]
    powList.append(a[i]/counts)
num = max(powList)
for o in range(0,22):
    if num == powList[o]:
        print("MOST CONSISTENT INVERTER IN PLANT 2 : ", source_list[o])

In [ ]:
# IRRADIATION VS DC POWER IN PLANT 1

df_plant1 = pd.merge(df1, df2, on='DATE_TIME', how='left')
plt.figure(figsize=(20,10))
plt.title("Irradiation vs DC Power Plant 1")
plt.scatter(df_plant1['IRRADIATION'], df_plant1['DC_POWER'], c='c', alpha=0.5)
plt.xlabel("Irradiation")
plt.ylabel("DC Power")
plt.grid()
plt.show()

In [ ]:
# IRRADIATION VS DC POWER IN PLANT 2

# PLANT 2
df_plant2 = pd.merge(df3, df4, on='DATE_TIME', how='left')
plt.figure(figsize=(20,10))
plt.title("Irradiation vs DC Power Plant 2")
sns.lineplot(x='IRRADIATION', y='DC_POWER', data=df_plant2, color='magenta')
plt.grid()
plt.show()

In [ ]:
#AMBIENT TEMPERATURE AGAINST DC POWER FOR PLANT 1 

df_plant1 = pd.merge(df1, df2, on='DATE_TIME', how='left')
plt.figure(figsize=(20,10))
plt.title("Ambient Temperature vs DC Power Plant 1")
plt.scatter(df_plant1['AMBIENT_TEMPERATURE'], df_plant1['DC_POWER'], c='c', alpha=0.5)
plt.xlabel("Ambient Temperature")
plt.ylabel("DC Power")
plt.grid()
plt.show()

In [ ]:
#AMBIENT TEMPERATURE AGAINST DC POWER FOR PLANT 2

df_plant2 = pd.merge(df3, df4, on='DATE_TIME', how='left')
plt.figure(figsize=(20,10))
plt.title("Ambient Temperature vs DC Power Plant 1")
plt.scatter(df_plant2['AMBIENT_TEMPERATURE'], df_plant2['DC_POWER'], c='b', alpha=0.5)
plt.xlabel("Ambient Temperature")
plt.ylabel("DC Power")
plt.grid()
plt.show()

In [ ]:
#COMPARING THE MEAN DC POWER FOR PLANT 1 and PLANT 2

labels= ['plant1', 'plant2']
values = [df1['DC_POWER'].mean(), df3['DC_POWER'].mean()]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(
    title_text="MEAN DC POWER COMPARISION")
fig.show()

In [ ]:
#COMPARING THE MEAN AC POWER FOR PLANT 1 and PLANT 2

labels= ['plant1', 'plant2']
values = [df1['AC_POWER'].mean(), df3['AC_POWER'].mean()]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(
    title_text="MEAN AC POWER COMPARISION")
fig.show()

In [ ]:
#Mean total yield 

labels= ['plant1', 'plant2']
values = [df1['TOTAL_YIELD'].mean(), df3['TOTAL_YIELD'].mean()]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(
    title_text="TOTAL YIELD COMPARISION")
fig.show()

In [ ]:
#DC POWER AGAINST TIME PLOT

fig = make_subplots(rows=2, cols=2, shared_yaxes=True)

fig.add_trace(go.Scatter(x=df1['DATE_TIME'], y=df1['DC_POWER'],name="Plant 1"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df3['DATE_TIME'], y=df3['DC_POWER'],name="Plant 2"),
              row=1, col=2)

fig.update_layout(height=1200, width=1200,
                  title_text="DC Power of both plants compared with date time")


fig.update_xaxes(title_text="Date Time", row=1, col=1)
fig.update_xaxes(title_text="Date Time", row=1, col=2)

fig.update_yaxes(title_text="DC Power", row=1, col=1)
fig.update_yaxes(title_text="DC Power", row=1, col=2)

fig.show()


In [ ]:
#AC POWER OF BOTH PLANTS AGAINST TIME 

fig = make_subplots(rows=2, cols=2, shared_yaxes=True)

fig.add_trace(go.Scatter(x=df1['DATE_TIME'], y=df1['AC_POWER'],name="Plant 1"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df3['DATE_TIME'], y=df3['AC_POWER'],name="Plant 2"),
              row=1, col=2)

fig.update_layout(height=1200, width=1200,
                  title_text="AC Power of both plants compared with date time")

fig.update_xaxes(title_text="Date Time", row=1, col=1)
fig.update_xaxes(title_text="Date Time", row=1, col=2)

fig.update_yaxes(title_text="AC Power", row=1, col=1)
fig.update_yaxes(title_text="AC Power", row=1, col=2)

fig.show()

In [ ]:
#DC POWER OF MOST CONSISTENT INVERTER IN PLANT 1
plant1m = pd.DataFrame({'SOURCE_KEY': df1['SOURCE_KEY'], 'TIME': df1['TIME'], 'DC_POWER': df1['DC_POWER']})
plant1m = plant1m.groupby(['SOURCE_KEY', 'TIME']).mean()
StartInv = int()
source_key_check = df1['SOURCE_KEY'].unique()

p = df1['DATE_TIME'].dt.time
uniqueTime = p.unique()
for o in range(0,24*4):
    uniqueTime[o] = str(uniqueTime[o])
TimeList = list()
for i in range(0,24*4, 4):
    TimeList.append(uniqueTime[i])
#SINCE EACH SOURCE KEY HAS 24*4 = 96 ROWS, Whichever Value of source_key_check the invertor is, it starts at the 96*(i+1) position of the new plant1m and ends at the 96*(i+1) + 95th position
for i in range(0,22):
    if source_key_check[i] == 'bvBOhCH3iADSZry':
        StartInv = 96*(i+1)
        
plt.figure(figsize = (21, 12))
plt.bar(uniqueTime,plant1m[StartInv:StartInv+96]['DC_POWER'], color = 'red')
plt.xticks(TimeList, rotation = 90)
plt.xlabel('TIME OF THE DAY')
plt.ylabel('DC POWER PRODUCED (in kW)')
plt.title('DC POWER OF THE MOST CONSISTENT INVERTER OF PLANT 1 ON AN AVERAGE DAY')
plt.show()

In [ ]:
#DC POWER OF MOST CONSISTENT INVERTER IN PLANT 2
plant1m = pd.DataFrame({'SOURCE_KEY': df3['SOURCE_KEY'], 'TIME': df3['TIME'], 'DC_POWER': df3['DC_POWER']})
plant1m = plant1m.groupby(['SOURCE_KEY', 'TIME']).mean()
StartInv = int()
source_key_check = df1['SOURCE_KEY'].unique()

p = df1['DATE_TIME'].dt.time
uniqueTime = p.unique()
for o in range(0,24*4):
    uniqueTime[o] = str(uniqueTime[o])
TimeList = list()
for i in range(0,24*4, 4):
    TimeList.append(uniqueTime[i])
#SINCE EACH SOURCE KEY HAS 24*4 = 96 ROWS, Whichever Value of source_key_check the invertor is, it starts at the 96*(i+1) position of the new plant1m and ends at the 96*(i+1) + 95th position
for i in range(0,22):
    if source_key_check[i] == 'bvBOhCH3iADSZry':
        StartInv = 96*(i+1)
        
plt.figure(figsize = (21, 12))
plt.bar(uniqueTime,plant1m[StartInv:StartInv+96]['DC_POWER'], color = 'green')
plt.xticks(TimeList, rotation = 90)
plt.xlabel('TIME OF THE DAY')
plt.ylabel('DC POWER PRODUCED (in kW)')
plt.title('DC POWER OF THE MOST CONSISTENT INVERTER OF PLANT 2 ON AN AVERAGE DAY')
plt.show()

In [ ]:
#COMPARING IRRADIATION OF WEATHER SENSOR PLANT 1 ON A DAILY BASIS AT DIFFERENT TIME DURATIONS IN A DAY

fig = px.bar(df2, x='DATE', y='IRRADIATION', color='IRRADIATION',
             labels={'Date':'Irradiation'}, height=600)
fig.show()

In [ ]:
#COMPARING IRRADIATION OF WEATHER SENSOR PLANT 2 ON A DAILY BASIS AT DIFFERENT TIME DURATIONS IN A DAY

fig = px.bar(df4, x='DATE', y='IRRADIATION', color='IRRADIATION',
             labels={'Date':'Irradiation'}, height=600)
fig.show()

In [ ]:
#IRRADIATION WRT TIME 

_, ax = plt.subplots(1, 1, figsize=(24, 10))

ax.plot(df2['HOUR'],
        df2['IRRADIATION'],
        marker='o',
        linestyle='',
        alpha=.5,
        ms=10,
        label='Irradiation With Time')
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Irradiation vs. Time')
plt.xlabel('Hour')
plt.ylabel('Irradiation')
plt.show()

In [ ]:
#COMPARING AMBIENT AND TEMPERATURE OF BOTH THE WEATHER SENSOR PLANTS
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(
        x=df2['AMBIENT_TEMPERATURE'],
        y=df2['MODULE_TEMPERATURE'],
        mode="markers+text",
        text=["", "", ""],
        name="Weather Plant 1",
        textposition="bottom center"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df4['AMBIENT_TEMPERATURE'],
        y=df4['MODULE_TEMPERATURE'],
        mode="markers+text",
        text=["", "", ""],
        name="Weather Plant 2",
        textposition="bottom center"
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Ambient Temperature", row=1, col=1)
fig.update_xaxes(title_text="Ambient Temperature", row=1, col=2)

fig.update_yaxes(title_text="Module Temperature", row=1, col=1)
fig.update_yaxes(title_text="Module Temperature", row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Ambeint VS Module Temperature of both plants")

fig.show()

In [ ]:
#Plot for DC Power vs. Module Temperature for Plant 1

_, ax = plt.subplots(1, 1, figsize=(18, 9))

ax.plot(df_plant1.MODULE_TEMPERATURE,
        df_plant1.DC_POWER,
        marker='o',
        linestyle='',
        alpha=.5,
        ms=10,
        label='DC POWER')

ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('DC Power vs. Module Temperature')
plt.xlabel('Module Temperature')
plt.ylabel('DC Power')
plt.show()

In [ ]:
#DC POWER VS MODULE TEMPERATURE FOR EVERY INVERTER IN PLANT 1
_, ax = plt.subplots(1, 1, figsize=(18, 9))

dates = df_plant1['DATE_x'].unique()
dates

date = dates[4]
inverters = df_plant1['SOURCE_KEY_x'].unique()

for inverter in inverters:

    data = df_plant1[(df_plant1['DATE_x']==date)&(df_plant1['SOURCE_KEY_x']==inverter)&(df_plant1['IRRADIATION']>0.1)]

    ax.plot(data.MODULE_TEMPERATURE,
                data.DC_POWER,
                marker='o',
                linestyle='',
                alpha=.5,
                ms=10,
                label=inverter
               )
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('DC Power vs. Module Temperature')
plt.xlabel('Module Temperature')
plt.ylabel('DC Power')
plt.show()

In [ ]:
#DC POWER VS AC POWER COMPARISION    
import matplotlib.pyplot as plt

#SS Plotting How DCPower and AC Power changing with time
_, ax = plt.subplots(1, 1, figsize=(18, 9))
ax.plot(df1.DATE_TIME,
        df1.DC_POWER.rolling(window=20).mean(),
        label='DC Power'
       )

ax.plot(df1.DATE_TIME,
        df1.AC_POWER.rolling(window=20).mean(),
        label='AC Power'
       )


ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('DC Power and AC Power')
plt.xlabel('Date and Time')
plt.ylabel('Power')
plt.show()    


In [ ]:
#Plotting DC Power and AC Power changing against time

_, ax = plt.subplots(1, 1, figsize=(18, 9))
ax.plot(df1.HOUR,
        df1.DC_POWER.rolling(window=24).mean(),
        label='DC Power'
       )

ax.plot(df1.HOUR,
        df1.AC_POWER.rolling(window=20).mean(),
        label='AC Power'
       )


ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('DC Power and AC Power Plant 1')
plt.xlabel('Time of Day')
plt.ylabel('Power')
plt.show()


In [ ]:
#Plotting DCPower and AC Power changing against time plant 2
_, ax = plt.subplots(1, 1, figsize=(18, 9))
ax.plot(df3.HOUR,
        df3.DC_POWER.rolling(window=24).mean(),
        label='DC Power'
       )

ax.plot(df3.HOUR,
        df3.AC_POWER.rolling(window=20).mean(),
        label='AC Power'
       )


ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('DC Power and AC Power Plant 2')
plt.xlabel('Time of Day')
plt.ylabel('Power')
plt.show()


In [ ]:
#COMPARING DAILY YIELD OF BOTH PLANTS 
fig = make_subplots(rows=2, cols=2, shared_yaxes=True)

fig.add_trace(go.Scatter(x=df1['DATE_TIME'], y=df1['DAILY_YIELD'],name="Plant 1"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df3['DATE_TIME'], y=df3['DAILY_YIELD'],name="Plant 2"),
              row=1, col=2)

fig.update_layout(height=1200, width=1200,
                  title_text="Daily Yield of both plants compared with date time")
fig.update_xaxes(title_text="Date Time", row=1, col=1)
fig.update_xaxes(title_text="Date Time", row=1, col=2)

fig.update_yaxes(title_text="Daily Yield", row=1, col=1)
fig.update_yaxes(title_text="Daily Yield", row=1, col=2)


fig.show()

In [ ]:
#Sunburst Plot of collected and uncollected data of total irradiation in weather sensors
plant1yes= (1758)
plant1no= (3264- 1758)

plant2yes= (1863)
plant2no= (3264- 1863)
totalir= (3264)
irvalues= {'total irradiation': [plant1yes, plant2yes, plant1no, plant2no ], 'plant' : ['plant1','plant2','plant1','plant2'], 'plant 1' : ['collected', 'collected', 'not collected',  'not collected']}
irdf= pd.DataFrame(data=irvalues)
irdf

In [ ]:
fig = px.sunburst(irdf, path=['plant', 'plant 1'], values='total irradiation')
fig.show()

In [ ]:
#Heat Map for Ambient temperature of plant 1
df_amb1 = pd.read_csv("../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")
df_amb1 = df_amb1.drop(['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'MODULE_TEMPERATURE', 'IRRADIATION'], axis=1)
df_amb1['DATE'] = df2['DATE'].dt.date
df_amb1['HOUR'] = df2['TIME'].dt.hour
df_amb1 = df_amb1.groupby(['HOUR', 'DATE']).mean()
df_amb1 = df_amb1.unstack(level=0)

x_labels = list()
for i in range(24):
    x_labels.append(i)
plt.figure(figsize=(20, 10))
plt.title("AMBIENT TEMPERATURE PLANT 1 HEATMAP")
sns.heatmap(df_amb1, cmap="Spectral", linewidth=0.3, annot=True)
plt.xticks(np.arange(24)+0.5, labels=x_labels)
plt.xticks(rotation=0)
plt.xlabel('HOURS')
plt.ylabel('DATE')
plt.show()

In [ ]:
#Heat Map for Ambient temperature of plant 2
df_amb2 = pd.read_csv("../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")
df_amb2 = df_amb2.drop(['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'MODULE_TEMPERATURE', 'IRRADIATION'], axis=1)
df_amb2['DATE'] = df2['DATE'].dt.date
df_amb2['HOUR'] = df2['TIME'].dt.hour
df_amb2 = df_amb2.groupby(['HOUR', 'DATE']).mean()
df_amb2 = df_amb2.unstack(level=0)

x_labels = list()
for i in range(24):
    x_labels.append(i)
plt.figure(figsize=(20, 10))
plt.title("AMBIENT TEMPERATURE PLANT 2 HEATMAP")
sns.heatmap(df_amb2, cmap="Spectral", linewidth=0.3, annot=True)
plt.xticks(np.arange(24)+0.5, labels=x_labels)
plt.xticks(rotation=0)
plt.xlabel('HOURS')
plt.ylabel('DATE')
plt.show()

In [ ]:
#FORECASTING 

In [ ]:
sense1  = df2.rename(columns={'DATE_TIME':'ds', 'AMBIENT_TEMPERATURE':'y'})
sense1.info()

In [ ]:

# Make the prophet model and fit on the data
op = fbprophet.Prophet(changepoint_prior_scale=0.25) 
op.fit(sense1)
op_forecast = op.make_future_dataframe(periods=96, freq='H')
# Make predictions
op_forecast = op.predict(op_forecast)

In [ ]:
op.plot(op_forecast, xlabel = 'Date', ylabel = 'AMBIENT TEMPERATURE ')
plt.title('AMBIENT TEMPERATURE Prediction Weather Plant 1')

In [ ]:
df4.info()

In [ ]:
sense2  = df4.rename(columns={'DATE_TIME':'ds', 'AMBIENT_TEMPERATURE':'y'})


# Make the prophet model and fit on the data
op = fbprophet.Prophet(changepoint_prior_scale=0.25) 
op.fit(sense2)
op_forecast = op.make_future_dataframe(periods=96, freq='H')
# Make predictions
op_forecast = op.predict(op_forecast)

op.plot(op_forecast, xlabel = 'Date', ylabel = 'AMBIENT TEMPERATURE ')
plt.title('AMBIENT TEMPERATURE Prediction Weather Plant 2')

In [ ]:
#Forecasting and Predicting Daily Yield of both plants

day_summary = df1.groupby('DATE').agg(DAILY_YIELD=('DAILY_YIELD',sum),DATE=('DATE',max))
day_summary
day_summary = day_summary.rename(columns={'DATE':'ds', 'DAILY_YIELD':'y'})

day_summary_2 = df3.groupby('DATE').agg(DAILY_YIELD=('DAILY_YIELD',sum),DATE=('DATE',max))
day_summary_2
day_summary_2 = day_summary_2.rename(columns={'DATE':'ds', 'DAILY_YIELD':'y'})


op = fbprophet.Prophet(changepoint_prior_scale=0.25)
op.fit(day_summary)

op3 = fbprophet.Prophet(changepoint_prior_scale=0.25)
op3.fit(day_summary_2)

forecast = op.make_future_dataframe(periods = 30, freq ='D')
forecast = op.predict(forecast)
forecast3 = op3.make_future_dataframe(periods = 30, freq ='D')
forecast3 = op3.predict(forecast3)


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat'],
    name="Plant 1 daily yield"
))

fig.add_trace(go.Scatter(
    x=forecast3['ds'],
    y=forecast3['yhat'],
    name="Plant 2 daily yield"
))

fig.show()

